![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/jupyter/training/english/entity-ruler/EntityRuler_Without_Storage.ipynb)

In [3]:
!wget https://setup.johnsnowlabs.com/colab.sh -O - | bash

This notebook uses useStorage parameter as false (default). So the annotator will serialize patterns file data with SparkML parameters when saving the model.

**We recommend using the default value (setUseStorage=False), as shown in this notebook since the results of our benchmarks reflect that this configuration is faster than setUseStorage=True**

In [4]:
import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.sql import SparkSession

In [ ]:
spark = sparknlp.start()

In [6]:
data = spark.createDataFrame([["Lord Eddard Stark was the head of House Stark. John Snow lives in Winterfell."]]).toDF("text")

In [7]:
data.show(truncate=False)

+-----------------------------------------------------------------------------+
|text                                                                         |
+-----------------------------------------------------------------------------+
|Lord Eddard Stark was the head of House Stark. John Snow lives in Winterfell.|
+-----------------------------------------------------------------------------+



# Keywords Patterns

In [8]:
import json

keywords = [
          {
            "label": "PERSON",
            "patterns": ["Jon", "John", "John Snow", "Jon Snow"]
          },
          {
            "label": "PERSON",
            "patterns": ["Eddard", "Eddard Stark"]
          },
          {
            "label": "LOCATION",
            "patterns": ["Winterfell"]
          },
         ]

with open('./keywords.json', 'w') as jsonfile:
    json.dump(keywords, jsonfile)

We are going to use a JSON file with the following format:

In [9]:
! cat ./person.json

cat: ./person.json: No such file or directory


When working with keywords, we DON'T need a pipeline with Tokenizer anymore.

In [10]:
document_assembler = DocumentAssembler().setInputCol("text").setOutputCol("document")
sentence_detector = SentenceDetector().setInputCols("document").setOutputCol("sentence")

entity_ruler = EntityRulerApproach() \
    .setInputCols(["sentence"]) \
    .setOutputCol("entity") \
    .setPatternsResource("./keywords.json")

In [11]:
pipeline = Pipeline(stages=[document_assembler, sentence_detector, entity_ruler])
pipeline_model = pipeline.fit(data)

In [12]:
pipeline_model.transform(data).select("entity").show(truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|entity                                                                                                                                                                                                        |
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|[{chunk, 5, 16, Eddard Stark, {entity -> PERSON, sentence -> 0}, []}, {chunk, 47, 55, John Snow, {entity -> PERSON, sentence -> 1}, []}, {chunk, 66, 75, Winterfell, {entity -> LOCATION, sentence -> 1}, []}]|
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [13]:
light_pipeline = LightPipeline(pipeline_model)

In [14]:
annotations = light_pipeline.fullAnnotate("Doctor John Snow lives in London, whereas Lord Commander Jon Snow lives in Castle Black")[0]
annotations.keys()

dict_keys(['document', 'sentence', 'entity'])

In [15]:
annotations.get('entity')

[Annotation(chunk, 7, 15, John Snow, {'entity': 'PERSON', 'sentence': '0'}),
 Annotation(chunk, 57, 64, Jon Snow, {'entity': 'PERSON', 'sentence': '0'})]

We can define an id field to identify entities and it supports JSON Lines format as the example below.

In [16]:
keywords = [
            {
              "id": "names-with-j",
              "label": "PERSON",
              "patterns": ["Jon", "John", "John Snow", "Jon Snow"]
            },
            {
              "id": "names-with-e",
              "label": "PERSON",
              "patterns": ["Eddard", "Eddard Stark"]
            },
            {
              "id": "locations",
              "label": "LOCATION",
              "patterns": ["Winterfell"]
            },
         ]

with open('./keywords.jsonl', 'w') as jsonlfile:
    for keyword in keywords:
      json.dump(keyword, jsonlfile)
      jsonlfile.write('\n')

In [17]:
! cat ./keywords.jsonl

{"id": "names-with-j", "label": "PERSON", "patterns": ["Jon", "John", "John Snow", "Jon Snow"]}
{"id": "names-with-e", "label": "PERSON", "patterns": ["Eddard", "Eddard Stark"]}
{"id": "locations", "label": "LOCATION", "patterns": ["Winterfell"]}


In [18]:
entity_ruler = EntityRulerApproach() \
    .setInputCols(["sentence"]) \
    .setOutputCol("entity") \
    .setPatternsResource("./keywords.jsonl", ReadAs.TEXT, options={"format": "JSONL"})

In [19]:
pipeline = Pipeline(stages=[document_assembler, sentence_detector, entity_ruler])
model = pipeline.fit(data)
model.transform(data).select("entity").show(truncate=False)

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|entity                                                                                                                                                                                                                                                                 |
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|[{chunk, 5, 16, Eddard Stark, {entity -> PERSON, sentence -> 0, id -> names-with-e}, []}, {chunk, 47, 55, John Snow, {entity -> PERSON, sentence -> 1, id -> names-with-j}, []}, {chunk, 66, 75, Winterfe

For the CSV file we use the following configuration:


In [20]:
with open('./keywords.csv', 'w') as csvfile:
    csvfile.write('PERSON|Jon\n')
    csvfile.write('PERSON|John\n')
    csvfile.write('PERSON|John Snow\n')
    csvfile.write('LOCATION|Winterfell')

In [21]:
! cat ./keywords.csv

PERSON|Jon
PERSON|John
PERSON|John Snow
LOCATION|Winterfell

In [22]:
entity_ruler_csv = EntityRulerApproach() \
    .setInputCols(["sentence"]) \
    .setOutputCol("entity") \
    .setPatternsResource("./keywords.csv", options={"format": "csv", "delimiter": "\\|"})

In [23]:
pipeline_csv = Pipeline(stages=[document_assembler, sentence_detector, entity_ruler_csv])
model_csv = pipeline_csv.fit(data)

In [24]:
model_csv.transform(data).select("entity").show(truncate=False)

+-----------------------------------------------------------------------------------------------------------------------------------------+
|entity                                                                                                                                   |
+-----------------------------------------------------------------------------------------------------------------------------------------+
|[{chunk, 47, 55, John Snow, {entity -> PERSON, sentence -> 1}, []}, {chunk, 66, 75, Winterfell, {entity -> LOCATION, sentence -> 1}, []}]|
+-----------------------------------------------------------------------------------------------------------------------------------------+



# Regex Patterns

Starting Spark NLP 4.2.0, regex patterns are defined at a more granular level, with each label. For example, we can have the JSON file below:

In [25]:
data = spark.createDataFrame([["The address is 123456 in Winterfell"]]).toDF("text")

In [26]:
patterns_string = """
[
  {
    "id": "id-regex",
    "label": "ID",
    "patterns": ["[0-9]+"],
    "regex": true
  },
  {
    "id": "locations-words",
    "label": "LOCATION",
    "patterns": ["Winterfell"],
    "regex": false
  }
]
"""
patterns_obj = json.loads(patterns_string)
with open('./patterns.json', 'w') as jsonfile:
    json.dump(patterns_obj, jsonfile)

In [27]:
!cat ./patterns.json

[{"id": "id-regex", "label": "ID", "patterns": ["[0-9]+"], "regex": true}, {"id": "locations-words", "label": "LOCATION", "patterns": ["Winterfell"], "regex": false}]

When defining a regex pattern, we need to define Tokenizer annotator in the pipeline

In [28]:
tokenizer = Tokenizer().setInputCols("sentence").setOutputCol("token")

In [29]:
regex_entity_ruler = EntityRulerApproach() \
    .setInputCols(["sentence", "token"]) \
    .setOutputCol("entity") \
    .setPatternsResource("./patterns.json")

In [30]:
regex_pipeline = Pipeline(stages=[document_assembler, sentence_detector, tokenizer, regex_entity_ruler])
regex_model = regex_pipeline.fit(data)

In [31]:
regex_model.transform(data).select("entity").show(truncate=False)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|entity                                                                                                                                                                   |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|[{chunk, 15, 20, 123456, {entity -> ID, id -> id-regex, sentence -> 0}, []}, {chunk, 25, 34, Winterfell, {entity -> LOCATION, sentence -> 0, id -> locations-words}, []}]|
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------+

